Notebook to backtest with backtrader the ML scores built for smart DCA on CW8

In [53]:
from importlib import reload
import pandas as pd
import backtrader as bt
import bt_ml_strategies as strat
reload(strat)

PATH_DATA_BT = "../../Data/backtest/"



In [4]:
df = pd.read_csv(PATH_DATA_BT+"DCA_CLOSE_1D_V1_lab_perf_21d_predict_BT.zip", sep=",",
                            index_col="OPEN_DATETIME", parse_dates=True, keep_default_na=True, low_memory=False)

In [17]:
prices=df[['open','high','low','close','volume','Predict','SL','TP']]
prices[['SL','TP']]=0
prices=prices['2010-04-01 00:00:00':]
prices.dropna(inplace=True)
prices.sort_index(inplace=True)
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3311 entries, 2010-04-01 to 2023-03-08
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   open     3311 non-null   float64
 1   high     3311 non-null   float64
 2   low      3311 non-null   float64
 3   close    3311 non-null   float64
 4   volume   3311 non-null   int64  
 5   Predict  3311 non-null   float64
 6   SL       3311 non-null   int64  
 7   TP       3311 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 232.8 KB


In [18]:
data = strat.SignalData(dataname=prices,timeframe=bt.TimeFrame.Days)#, compression=30)

In [54]:
dca_params = {'score_buy': 0.01, 'score_sell': -50, 'size_buy': 2500, 'size_sell': 5000}
cerebro = bt.Cerebro(cheat_on_open=True)
cerebro.addstrategy(strat.BtMlDcaStrategy,**dca_params)
cerebro.adddata(data)
cerebro.broker.setcash(1000000.0)
cerebro.broker.setcommission(commission=0.0025)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")

print(f"Starting portfolio:{cerebro.broker.getvalue()}")
backtest_result=cerebro.run()
print(f"Ending portfolio:{cerebro.broker.getvalue()}")

Starting portfolio:1000000.0
2010-04-06 00:00:00, TRACE 1 --- self.top_buy=True, self.prev_mt=4, dt.month=4
2010-04-30 00:00:00, TRACE SIZE --- size_max=9020, size_ord=22, Predict=0.01101
2010-04-30 00:00:00, BUY CREATED --- Size: size_ord=22, Cash: 1000000.00
2010-04-30 00:00:00, BUY EXECUTED 829 --- Price: 110.86, Cost: 2438.92,Commission: 6.10, Size:   22.0000
2010-05-03 00:00:00, TRACE 2 --- self.top_buy=True, self.prev_mt=5, dt.month=5, ptf=2438.92
2010-05-03 00:00:00, TRACE SIZE --- size_max=8886, size_ord=22, Predict=0.01126
2010-05-03 00:00:00, BUY CREATED --- Size: size_ord=22, Cash: 997554.98
2010-05-03 00:00:00, BUY EXECUTED 830 --- Price: 112.25, Cost: 2469.50,Commission: 6.17, Size:   22.0000
2010-06-01 00:00:00, TRACE 2 --- self.top_buy=True, self.prev_mt=6, dt.month=6, ptf=4908.42
2010-06-01 00:00:00, TRACE SIZE --- size_max=9261, size_ord=23, Predict=0.0193
2010-06-01 00:00:00, BUY CREATED --- Size: size_ord=23, Cash: 995079.31
2010-06-01 00:00:00, BUY EXECUTED 831 --- 